In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
# load the dataset
df = pd.read_csv('data/papers.csv').head(500)
df.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          500 non-null    int64 
 1   year        500 non-null    int64 
 2   title       500 non-null    object
 3   event_type  0 non-null      object
 4   pdf_name    500 non-null    object
 5   abstract    500 non-null    object
 6   paper_text  500 non-null    object
dtypes: int64(2), object(5)
memory usage: 27.5+ KB


In [4]:
print("{} abstracts are missing".format(df[df['abstract']=='Abstract Missing']['abstract'].count()))

500 abstracts are missing


In [5]:
import pprint
sample = 100
pprint.pprint("TITLE:{}".format(df['title'][sample]))
pprint.pprint("ABSTRACT:{}".format(df['abstract'][sample]))
pprint.pprint("FULL TEXT:{}".format(df['paper_text'][sample][:1000]))

('TITLE:Beating a Defender in Robotic Soccer: Memory-Based Learning of a '
 'Continuous Function')
'ABSTRACT:Abstract Missing'
('FULL TEXT:Beating a Defender in Robotic Soccer:\n'
 'Memory-Based Learning of a Continuous\n'
 'FUnction\n'
 'Peter Stone\n'
 'Department of Computer Science\n'
 'Carnegie Mellon University\n'
 'Pittsburgh, PA 15213\n'
 '\n'
 'Manuela Veloso\n'
 'Department of Computer Science\n'
 'Carnegie Mellon University\n'
 'Pittsburgh, PA 15213\n'
 '\n'
 'Abstract\n'
 "Learning how to adjust to an opponent's position is critical to\n"
 'the success of having intelligent agents collaborating towards the\n'
 'achievement of specific tasks in unfriendly environments. This paper '
 'describes our work on a Memory-based technique for to choose\n'
 'an action based on a continuous-valued state attribute indicating\n'
 'the position of an opponent. We investigate the question of how an\n'
 'agent performs in nondeterministic variations of the training situations. '
 'Our exper

This dataset contains 7 columns: id, year, title, even_type, pdf_name, abstract and paper_text. We are mostly interested in the paper_text which include both title and abstract.

## Pre-processing

In [6]:
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
##Creating a list of custom stopwords
new_words = ["fig","figure","image","sample","using", 
             "show", "result", "large", 
             "also", "one", "two", "three", 
             "four", "five", "seven","eight","nine"]
stop_words = list(stop_words.union(new_words))

def pre_process(text):
    # lowercase
    text=text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    ##Convert to list from string
    text = text.split()
    # remove stopwords
    text = [word for word in text if word not in stop_words]
    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]
    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    return ' '.join(text)

In [7]:
%%time
docs = df['paper_text'].apply(lambda x:pre_process(x))

CPU times: user 5.38 s, sys: 51.2 ms, total: 5.43 s
Wall time: 5.43 s


In [8]:
docs[1][0:103]

'mean field theory layer visual cortex application artificial neural network christopher scofield center'

## 1.TF-IDF and Scikit-learn

Based on the tutorial of [Kavita Ganesan](https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb)

TF-IDF stands for Text Frequency Inverse Document Frequency. The importance of each word increases proportionally to the number of times a word appears in the document (Text Frequency - TF) but is offset by the frequency of the word in the corpus (Inverse Document Frequency - IDF). Using the tf-idf weighting scheme, the keywords are the words with the higherst TF-IDF score.

### 1.1 CountVectorizer to create a vocabulary and generate word counts

In [9]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
#docs = docs.tolist()
#create a vocabulary of words, 
cv=CountVectorizer(max_df=0.95,         # ignore words that appear in 95% of documents
                   max_features=10000,  # the size of the vocabulary
                   ngram_range=(1,3)    # vocabulary contains single words, bigrams, trigrams
                  )
word_count_vector=cv.fit_transform(docs)

CPU times: user 4.34 s, sys: 87.2 ms, total: 4.43 s
Wall time: 4.43 s


### 1.2 TfidfTransformer to Compute Inverse Document Frequency (IDF)

In [10]:
%%time
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

CPU times: user 1.69 ms, sys: 170 µs, total: 1.86 ms
Wall time: 1.32 ms


Once we have our IDF computed, we are now ready to compute TF-IDF and extract the top keywords.

In [11]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [12]:
# get feature names
feature_names=cv.get_feature_names()

def get_keywords(idx, docs):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords, df):
    # now print the results
    print("\n=====Title=====")
    print(df['title'][idx])
    print("\n=====Abstract=====")
    print(df['abstract'][idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [13]:
idx=100
keywords=get_keywords(idx, docs)
# print_results(idx,keywords, df)

In [14]:
keywords

{'action': 0.091,
 'agent': 0.393,
 'defender': 0.584,
 'learning': 0.105,
 'mem': 0.206,
 'memory': 0.357,
 'nondeterministic': 0.099,
 'speed': 0.133,
 'stone': 0.121,
 'veloso': 0.165}

I am not happy with result. I would like to add a filter that will remove similar keywords, or short keywords inside of complex ones. For instance, non-negative matrix factorization meets us 5 time: non negative matrix, negative matrix, nmf, matrix factorization, matrix. Adding a 4-grams does not change the situation. Similar keywords appears due to the fact that TF-IDF does not take into account the context, the keywords importance comes only from their frequencies relationship. Thus, TF-IDF is a quick, intuitive, but not the best way to extract keywords from the text. Let's look at other ways.

## 2. Gensim implementation of TextRank summarization algorithm

Gensim is a free Python library designed to automatically extract semantic topics from documents. The gensim implementation is based on the popular TextRank algorithm. 

[Documentation](https://radimrehurek.com/gensim/summarization/keywords.html)

[Tutorial](https://rare-technologies.com/text-summarization-with-gensim/)

### 2.1 Small text

In [14]:
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /home/waves8/anaconda3/envs/thy_env/lib/python3.6/site-packages (3.8.3)


In [15]:
import gensim
text = "Non-negative matrix factorization (NMF) has previously been shown to " + \
"be a useful decomposition for multivariate data. Two different multiplicative " + \
"algorithms for NMF are analyzed. They differ only slightly in the " + \
"multiplicative factor used in the update rules. One algorithm can be shown to " + \
"minimize the conventional least squares error while the other minimizes the  " + \
"generalized Kullback-Leibler divergence. The monotonic convergence of both  " + \
"algorithms can be proven using an auxiliary function analogous to that used " + \
"for proving convergence of the Expectation-Maximization algorithm. The algorithms  " + \
"can also be interpreted as diagonally rescaled gradient descent, where the  " + \
"rescaling factor is optimally chosen to ensure convergence."
gensim.summarization.keywords(text, 
         ratio=0.5,               # use 50% of original text
         words=None,              # Number of returned words
         split=True,              # Whether split keywords
         scores=False,            # Whether score of keyword
         pos_filter=('NN', 'JJ'), # Part of speech (nouns, adjectives etc.) filters
         lemmatize=True,         # If True - lemmatize words
         deacc=True)              # If True - remove accentuation

['factor',
 'convergence',
 'rescaling',
 'multiplicative',
 'function',
 'kullback',
 'gradient',
 'algorithm',
 'matrix',
 'useful decomposition',
 'data',
 'multivariate',
 'squares']

In [16]:
print("SUMMARY: ", gensim.summarization.summarize(text,
                                                  ratio = 0.5,
                                                  split = True))

SUMMARY:  ['Non-negative matrix factorization (NMF) has previously been shown to be a useful decomposition for multivariate data.', 'Two different multiplicative algorithms for NMF are analyzed.', 'They differ only slightly in the multiplicative factor used in the update rules.']


### 2.2 Large text

In [17]:
def get_keywords_gensim(idx, docs):
    
    keywords=gensim.summarization.keywords(docs[idx], 
                                  ratio=None, 
                                  words=10,         
                                  split=True,             
                                  scores=False,           
                                  pos_filter=None, 
                                  lemmatize=True,         
                                  deacc=True)              
    
    return keywords

def print_results_gensim(idx,keywords, df):
    # now print the results
    print("\n=====Title=====")
    print(df['title'][idx])
    print("\n=====Abstract=====")
    print(df['abstract'][idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k)

In [18]:
idx=100
keywords=get_keywords_gensim(idx, docs)
print_results_gensim(idx,keywords, df)


=====Title=====
Beating a Defender in Robotic Soccer: Memory-Based Learning of a Continuous Function

=====Abstract=====
Abstract Missing

===Keywords===
learn
memory
agent
defender
positive
resulting
experience
representing
train
action


The keywords highlight the main point , but still miss valuable information

## 3. Python implementation of the Rapid Automatic Keyword Extraction algorithm (RAKE) using NLTK

[Documentation](https://github.com/csurfer/rake-nltk)

### Setup using pip

In [19]:
!pip install rake-nltk

### or directly from the repository

In [20]:
# !git clone https://github.com/csurfer/rake-nltk.git
# !python rake-nltk/setup.py install

### 3.1 Small text

In [21]:
text = "Non-negative matrix factorization (NMF) has previously been shown to " + \
"be a useful decomposition for multivariate data. Two different multiplicative " + \
"algorithms for NMF are analyzed. They differ only slightly in the " + \
"multiplicative factor used in the update rules. One algorithm can be shown to " + \
"minimize the conventional least squares error while the other minimizes the  " + \
"generalized Kullback-Leibler divergence. The monotonic convergence of both  " + \
"algorithms can be proven using an auxiliary function analogous to that used " + \
"for proving convergence of the Expectation-Maximization algorithm. The algorithms  " + \
"can also be interpreted as diagonally rescaled gradient descent, where the  " + \
"rescaling factor is optimally chosen to ensure convergence."

In [22]:
from rake_nltk import Rake
r = Rake()
r.extract_keywords_from_text(text)
r.get_ranked_phrases_with_scores()[:10]

[(16.0, 'diagonally rescaled gradient descent'),
 (16.0, 'conventional least squares error'),
 (14.0, 'two different multiplicative algorithms'),
 (9.0, 'negative matrix factorization'),
 (9.0, 'auxiliary function analogous'),
 (8.0, 'multiplicative factor used'),
 (4.5, 'rescaling factor'),
 (4.0, 'useful decomposition'),
 (4.0, 'update rules'),
 (4.0, 'proving convergence')]

Wow! We see well interbretable machine learning terminology! But why diagonally rescaled gradient descent is more important than negative matrix factorization? 

### 3.2 Large Text

In [23]:
def get_keywords_rake(idx, docs, n=10):
    # Uses stopwords for english from NLTK, and all puntuation characters by default
    r = Rake()
    
    # Extraction given the text.
    r.extract_keywords_from_text(docs[idx][1000:2000])
    
    # To get keyword phrases ranked highest to lowest.
    keywords = r.get_ranked_phrases()[0:n]
    
    return keywords

def print_results(idx,keywords, df):
    # now print the results
    print("\n=====Title=====")
    print(df['title'][idx])
    print("\n=====Abstract=====")
    print(df['abstract'][idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k)

idx=100
keywords = get_keywords_rake(idx, docs, n=10)
print_results(idx, keywords, df)


=====Title=====
Beating a Defender in Robotic Soccer: Memory-Based Learning of a Continuous Function

=====Abstract=====
Abstract Missing

===Keywords===
ial environment assume agent specific goal achieve conduct investigation framework team agent compete game robotic soccer real system model car remotely controlled board computer development research currently conducted simulator physical system simulator real world system based closely system designed laboratory computationalintelligence university british columbia sahota sahota simulator facilitates control number car ball within designated playing area care taken ensure simulator model real world response friction conserva memory based learning continuous function tion momentum etc closely possible show simulator graphic graphic view simulator initial position experiment paper teammate black remains stationary defender white move small circle different speed ball move either directly towards goal towards teammate position ball rep

Oups! Something goes wrong! Algorithm does not work for the preprocessed text without punctuations. Let's treat the raw text.

In [24]:
idx=100
keywords = get_keywords_rake(idx, df['paper_text'], n=10)
print_results(idx, keywords, df)


=====Title=====
Beating a Defender in Robotic Soccer: Memory-Based Learning of a Continuous Function

=====Abstract=====
Abstract Missing

===Keywords===
model cars remotely controlled
machine learning perspective
designated playing area
multiple agents collaborating
research works towards
intelligent agents
agents compete
world system
systems designed
specific goal


Presented implementation works well on sentences, but it is not flexible enough for large text. However, those who are interested in RANK can expand the capabilities of this code to their needs. We will consider next options.

## 4. Yet Another Keyword Extractor (Yake)

[Documentation](https://github.com/LIAAD/yake)

In [25]:
import yake

def get_keywords_yake(idx, docs):
    y = yake.KeywordExtractor(lan='en',          # language
                             n = 3,              # n-gram size
                             dedupLim = 0.9,     # deduplicationthresold
                             dedupFunc = 'seqm', #  deduplication algorithm
                             windowsSize = 1,
                             top = 10,           # number of keys
                             features=None)           
    
    keywords = y.extract_keywords(text)
    return keywords

idx=100
keywords = get_keywords_yake(idx, docs[idx])
print_results(idx, keywords, df)


=====Title=====
Beating a Defender in Robotic Soccer: Memory-Based Learning of a Continuous Function

=====Abstract=====
Abstract Missing

===Keywords===
('non-negative matrix factorization', 0.0041066275750552455)
('non-negative matrix', 0.026529705128479162)
('matrix factorization', 0.026529705128479162)
('multivariate data', 0.026529705128479162)
('decomposition for multivariate', 0.03127464030655176)
('nmf', 0.06699743201311577)
('nmf are analyzed', 0.11148839518508058)
('algorithms', 0.13323194577601624)
('previously been shown', 0.1372005684192386)
('non-negative', 0.1484061535685674)


Key phrases are repeated, and the text needs pre-processing to remove stop words

## 5. Keyphrases extraction using pke

`pke` an open source python-based keyphrase extraction toolkit. It provides an end-to-end keyphrase extraction pipeline in which each component can be easily modified or extended to develop new models.

`pke` currently implements the following keyphrase extraction models:

* Unsupervised models
  * Statistical models
    * TfIdf [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#tfidf)]
    * KPMiner [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#kpminer), [article by (El-Beltagy and Rafea, 2010)](http://www.aclweb.org/anthology/S10-1041.pdf)]
    * YAKE [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#yake), [article by (Campos et al., 2020)](https://doi.org/10.1016/j.ins.2019.09.013)]
  * Graph-based models
    * TextRank [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#textrank), [article by (Mihalcea and Tarau, 2004)](http://www.aclweb.org/anthology/W04-3252.pdf)]
    * SingleRank  [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#singlerank), [article by (Wan and Xiao, 2008)](http://www.aclweb.org/anthology/C08-1122.pdf)]
    * TopicRank [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#topicrank), [article by (Bougouin et al., 2013)](http://aclweb.org/anthology/I13-1062.pdf)]
    * TopicalPageRank [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#topicalpagerank), [article by (Sterckx et al., 2015)](http://users.intec.ugent.be/cdvelder/papers/2015/sterckx2015wwwb.pdf)]
    * PositionRank [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#positionrank), [article by (Florescu and Caragea, 2017)](http://www.aclweb.org/anthology/P17-1102.pdf)]
    * MultipartiteRank [[documentation](https://boudinfl.github.io/pke/build/html/unsupervised.html#multipartiterank), [article by (Boudin, 2018)](https://arxiv.org/abs/1803.08721)]
* Supervised models
  * Feature-based models
    * Kea [[documentation](https://boudinfl.github.io/pke/build/html/supervised.html#kea), [article by (Witten et al., 2005)](https://www.cs.waikato.ac.nz/ml/publications/2005/chap_Witten-et-al_Windows.pdf)]
    * WINGNUS [[documentation](https://boudinfl.github.io/pke/build/html/supervised.html#wingnus), [article by (Nguyen and Luong, 2010)](http://www.aclweb.org/anthology/S10-1035.pdf)]


In [26]:
# !pip install git+https://github.com/boudinfl/pke.git

In [27]:
import pke

### 5.1  SingleRank

This model is an extension of the TextRank model that uses the number of co-occurrences to weigh edges in the graph.

In [29]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a SingleRank extractor.
extractor = pke.unsupervised.SingleRank()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None)

# 3. select the longest sequences of nouns and adjectives as candidates.
extractor.candidate_selection(pos=pos)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk. In the graph, nodes are words of
#    certain part-of-speech (nouns and adjectives) that are connected if
#    they occur in a window of 10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

idx = 100
# now print the results
print("\n=====Title=====")
print(df['title'][idx])
print("\n=====Abstract=====")
print(df['abstract'][idx])
print("\n===Keywords===")
for k in keyphrases:
    print(k[0])


=====Title=====
Beating a Defender in Robotic Soccer: Memory-Based Learning of a Continuous Function

=====Abstract=====
Abstract Missing

===Keywords===
different multiplicative algorithms
non - negative matrix factorization
conventional least squares error
multiplicative factor
monotonic convergence
algorithms
other minimizes
maximization algorithm
auxiliary function analogous
rescaling factor


### 5.2 TopicRank

In [35]:
text = """
We’re living through a fascinating era of rapid change for the blockbuster movie model. America producers, eager to get their $200 million movies into the lucrative Chinese market, are increasingly looking for Chinese production partners, shooting in Chinese locations, and adding China-friendly characters and plotlines to American movies, even including extra scenes just for the Chinese cuts of films. But simultaneously, China and other countries are moving toward the blockbuster model themselves, creating homegrown films that don’t need to involve American partners at all.
And just as American films attempt to find paydays in foreign markets, foreign blockbusters are coming to America. The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.
The film, based on a short story by Three-Body Problem author Cixin Liu, lays out a crisis of unprecedented proportions: the sun has become unstable, and within a hundred years, it will expand to consume Earth. Within 300, the entire solar system will be gone. Earth’s governments rally and unite to face the problem, and come up with a novel solution: they speckle the planet with 10,000 gigantic jets, and blast it out of its orbit and off on a hundred-generation journey to a new home 4.2 light-years away. The idea is to use Jupiter’s gravitational well to pick up speed for the trip, but a malfunction of the Earth Engine system leaves the planet caught in Jupiter’s gravity, and gradually being pulled toward destruction. A frantic group of workers have to scramble to reactivate the jets and correct the Earth’s course.
The action takes place in two arenas simultaneously. On the Earth’s frigid surface, self-proclaimed genius Liu Qi (Qu Chuxiao) and his younger adopted sister Han Duoduo (Zhao Jinmai) get roped into the rescue efforts after they run away from home. Han is just curious to see the planet’s surface — most of humanity now lives in crowded underground cities, and the surface is for workers only — but Liu Qi is nursing a deeper grudge against his astronaut father Liu Peiqiang (longtime martial-arts movie star Wu Jing) and grandfather (Ng Man-tat, whom Western audiences might recognize from Stephen Chow’s Shaolin Soccer). When Liu Qi was a child, his father moved to a newly-built international space station, designed to move ahead of Earth as a guide and pathfinder. Now an adult, Liu Qi feels his father abandoned him, and wants to strike out independently.
Meanwhile, on the space station, Liu Peiqiang is ironically a day away from completing his 17-year tour of duty and returning to Earth and his family when the crisis hits. The station’s artificial intelligence, MOSS, insists on putting the station’s personnel in hibernation to save energy, but Liu Peiqiang realizes the computer has a secret agenda, and he and a Russian cosmonaut set out to defy it.
The entire space plot may feel suspiciously familiar to American audiences, who have a strong emotional touchstone when it comes to a calm-voiced computer in space telling a desperate astronaut that it can’t obey his orders, even when human lives are on the line, because it has orders of its own. MOSS even looks something like the HAL 9000 from 2001: A Space Odyssey: it’s represented as a red light on a gimbled panel, like a single unblinking, judgmental red eye. But a good deal of Liu Peiqiang’s space adventure also plays out like a sequence from Alfonso Cuarón’s 2013 Oscar-winner Gravity, with dizzying sequences of astronauts trying to navigate clouds of debris and find handholds on a treacherous moving station while tumbling through space.
Meanwhile, the Earthside half of the mission resembles nothing so much as the 2003 nonsense-thriller The Core, about a team trying to drill their way to the center of the Earth to set the planet’s core spinning again. Liu Qi and Han pick up a few distinctive allies along the way, including biracial Chinese-Australian gadabout Tim (viral video star Mike Sui), but mostly, the characters are drawn as blandly and broadly as in any American action movie, and a fair number of them get killed along the journey without ever having developed enough personality for audiences to feel the loss.
Pretty much any flaw The Wandering Earth can claim — flashy action scenes without much substance, a marked bent toward sticky sentimentality, an insistently pushy score that demands emotional response from the audience at every given moment — are familiar flaws from past blockbusters. Where the film really stands out, though, is in its eye for grandiose spectacle. Director Frant Gwo gives the film a surprising stateliness, especially in the scenes of the mobile Earth wandering the cosmos, wreathed in tiny blue jets that leave eerie space-contrails behind. His attention to detail is marvelous — in scenes where characters stand on Earth’s surface, contemplating Jupiter’s malicious beauty, the swirling colors of the Great Red Spot are clearly visible in reflections in their suit helmets.
No matter how familiar the plot beats feel, that level of attention not just to functional special effects, but to outright beauty, makes The Wandering Earth memorable. Not every CGI sequence is aesthetically impeccable — sequences like a vehicle chase through a frozen Shanghai sometimes look brittle and false. But everything having to do with Jupiter, Earth as seen from space, and the space station subplot is visually sumptuous. This is frequently a gorgeously rendered film, with an emphasis on intimidating space vistas that will look tremendous on IMAX screens.
And while the constant attempts to flee the destructive power of changing weather have their own echoes in past films, from The Day After Tomorrow to 2012, Gwo mostly keeps the action tight and propulsive. The Wandering Earth is frequently breathless, though the action occasionally gets a little muddled in editing. At times, particularly on the surface scenes where everyone is wearing identical pressure suits, it can be easy to lose track of which character is where. It’s often easy to feel that Gwo cares more about the collective rescue project than about any individual character — potentially a value that will work better for Chinese audiences than American viewers, who are looking for a single standout hero to root for.
But the film’s biggest strengths are in its quieter moments, where Gwo takes the time to contemplate Jupiter’s gravity well slowly deepening its pull on Earth’s atmosphere, or Liu Qi staring up, awestruck, at the gas giant dwarfing his home. In those chilly sequences, the film calls back to an older tradition of slower science fiction, in epic-scale classics like 1951’s When Worlds Collide or 1956’s Forbidden Planet. The interludes are brief, but they’re a welcome respite from chase sequences and destruction.
The Wandering Earth gets pretty goofy at times, with jokes about Tim’s heritage, or Liu Qi’s inexperienced driving and overwhelming arrogance, or with high-speed banter over an impossibly long technical manual that no one has time to digest in the middle of an emergency. At times, the humor is even a little dry, as when MOSS responds to Liu Peiqiang’s repeated rebellions with a passive-aggressive “Will all violators stop contact immediately with Earth?” But Gwo finds time for majesty as well, and makes a point of considering the problem on a global scale, rather than just focusing on the few desperate strivers who’ve tied the Earth’s potential destruction into their own personal issues.
Much like the Russian space blockbuster Salyut-7 was a fascinating look into the cultural differences between American films and their Russian equivalents, The Wandering Earth feels like a telling illustration of the similarities and differences between Chinese and American values. Gwo’s film is full of images and moments that will be familiar to American audiences, and it has an equally familiar preoccupation with the importance of family connections, and the nobility of sacrifice. But it also puts a strong focus on global collective action, on the need for international cooperation, and for the will of the group over the will of the individual.
None of these things will be inherently alien to American viewers, who may experience The Wandering Earth as a best-of mash-up of past science fiction films, just with less-familiar faces in the lead roles. But as China gets into the action-blockbuster business, it’ll continue to be fascinating to see how the country brings its own distinctive voices and talents into a global market. The Wandering Earth feels like the same kind of projects American filmmakers are making — accessible, thrill-focused, and at least somewhat generic, in an attempt to go down easy with any audience. But there’s enough specific personality in it to point to a future of more nationally inflected blockbusters. Once every country is making would-be international crossovers, the strongest appeal may come from the most distinctive, personal visions with the most to say about the cultures they come from.
"""

In [36]:
import string
from nltk.corpus import stopwords

# 1. create a TopicRank extractor.
extractor = pke.unsupervised.TopicRank()

# 2. load the content of the document.
extractor.load_document(input=text)

# 3. select the longest sequences of nouns and adjectives, that do
#    not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN', 'ADJ'}
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')
extractor.candidate_selection(pos=pos, stoplist=stoplist)

# 4. build topics by grouping candidates with HAC (average linkage,
#    threshold of 1/4 of shared stems). Weight the topics using random
#    walk, and select the first occuring candidate from each topic.
extractor.candidate_weighting(threshold=0.74, method='average')

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

idx = 100
# now print the results
print("\n=====Title=====")
print(df['title'][idx])
print("\n=====Abstract=====")
print(df['abstract'][idx])
print("\n===Keywords===")
for k in keyphrases:
    print(k[0])


=====Title=====
Beating a Defender in Robotic Soccer: Memory-Based Learning of a Continuous Function

=====Abstract=====
Abstract Missing

===Keywords===
wandering earth
films
entire space plot
genius liu qi
chinese production partners
times
sequence
director frant gwo
action
international space station


In [38]:
keyphrases

[('wandering earth', 0.04454298297904837),
 ('films', 0.026726986229525875),
 ('entire space plot', 0.01654283261399369),
 ('genius liu qi', 0.015725128443376625),
 ('chinese production partners', 0.015229488820999334),
 ('times', 0.014073050308177645),
 ('sequence', 0.013950319757885715),
 ('director frant gwo', 0.013702145983472108),
 ('action', 0.013037147499461419),
 ('international space station', 0.01283682037083518)]